# streamfunction_vorticity_nonlinear_solvers

> Solvers based on nonlinear optimization for the full Navier-Stokes equations in the streamfunction-vorticity form

In [ ]:
#| default_exp streamfunction_vorticity_nonlinear

# Imports

In [ ]:
# Autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from uom_project import poisson_solvers, streamfunction_vorticity_newton

import numpy as np
import scipy
from scipy import sparse

In [ ]:
from functools import partial

from fastcore.test import test_eq, test_close

# Nonlinear solvers

## Root

In [ ]:
#| export

# Given the vorticity, solve the Poisson eqn. to find the streamfunction
def get_standard_basis_vector(size, i):
    vec = np.zeros((size, ))
    vec[i] = 1.0
    
    return vec


def make_get_jacobian(f):
    def get_jacobian(x, Re, kernel_matrix):
        N = int(np.sqrt(x.shape[0] // 2 + 1))
        h = 1 / N

        f_evaluated = f(x=x, Re=Re, kernel_matrix=kernel_matrix)

        return np.vstack([(
                f(
                    x=x + h*get_standard_basis_vector(size=x.shape[0], i=i),
                    Re=Re, kernel_matrix=kernel_matrix
                ) -
                f_evaluated
            ) for i in range(x.shape[0])
        ]).T

    return get_jacobian


def f(x, Re, kernel_matrix, U_wall_top):
    N = int(np.sqrt(x.shape[0] // 2 + 1))
    h = 1 / N

    psi = x[:(N-1)**2]
    w_left   = x[(N-1)**2 + 0*(N-1) : (N-1)**2 + 1*(N-1)]
    w_right  = x[(N-1)**2 + 1*(N-1) : (N-1)**2 + 2*(N-1)]
    w_bottom = x[(N-1)**2 + 2*(N-1) : (N-1)**2 + 3*(N-1)]
    w_top    = x[(N-1)**2 + 3*(N-1) : (N-1)**2 + 4*(N-1)]
    w_middle = x[(N-1)**2 + 4*(N-1) :]

    # Calculate the equations coming from the Poisson equation
    f_poisson = kernel_matrix @ psi
    f_poisson = f_poisson + h ** 2 * w_middle

    psi = psi.reshape(N-1, N-1)

    # Calculate contributions coming from the vorticity transport equation
    w_middle = w_middle.reshape(N-1, N-1)
    
    # Calculate the sides first
    # y = 0, U_wall = 0
    f_w_bottom = h ** 2 * (w_middle[:, 0] + 3 * w_bottom) + 8 * psi[:, 0]
    # y = 1, U_wall is known here
    f_w_top = h ** 2 * (w_middle[:, -1] + 3 * w_top) + 8 * (
        h * U_wall_top + psi[:, -1]
    )
    # x = 0
    f_w_left = h ** 2 * (w_middle[0, :] + 3 * w_left) + 8 * psi[0, :]
    # x = 1
    f_w_right = h ** 2 * (w_middle[-1, :] + 3 * w_right) + 8 * psi[-1, :]

    f_w_middle = -4 * w_middle
    f_w_middle[:-1, :] += w_middle[1:, :]
    f_w_middle[-1:, :] += w_right
    f_w_middle[1:, :] += w_middle[:-1, :]
    f_w_middle[:1, :] += w_left
    f_w_middle[:, :-1] += w_middle[:, 1:]
    f_w_middle[:, -1] += w_top
    f_w_middle[:, 1:] += w_middle[:, :-1]
    f_w_middle[:, 0] += w_bottom

    f_w_middle[1:-1, 1:-1] += Re * (
        (psi[2:, 1:-1] - psi[:-2, 1:-1]) * (w_middle[1:-1, 2:] - w_middle[1:-1, :-2]) -
        (psi[1:-1, 2:] - psi[1:-1, :-2]) * (w_middle[2:, 1:-1] - w_middle[:-2, 1:-1])
    ) / 4
    f_w_middle[:1, 1:-1] += Re * (
        psi[1, 1:-1] * (w_middle[0, 2:] - w_middle[0, :-2]) -
        (psi[0, 2:] - psi[0, :-2]) * (w_middle[1, 1:-1] - w_left[1:-1])
    ) / 4
    f_w_middle[-1:, 1:-1] -= Re * (
        psi[-2, 1:-1] * (w_middle[-1, 2:] - w_middle[-1, :-2]) +
        (psi[-1, 2:] - psi[-1, :-2]) * (w_right[1:-1] - w_middle[-2, 1:-1])
    ) / 4
    f_w_middle[1:-1, 0] += Re * (
        (psi[2:, 0] - psi[:-2, 0]) * (w_middle[1:-1, 1] - w_bottom[1:-1]) -
        psi[1:-1, 1] * (w_middle[2:, 0] - w_middle[:-2, 0])
    ) / 4
    f_w_middle[1:-1, -1] += Re * (
        (psi[2:, -1] - psi[:-2, -1]) * (w_top[1:-1] - w_middle[1:-1, -2]) +
        psi[1:-1, -2] * (w_middle[2:, -1] - w_middle[:-2, -1])
    ) / 4
    f_w_middle[0, 0] += Re * (
        psi[1, 0] * (w_middle[0, 1] - w_bottom[0]) -
        psi[0, 1] * (w_middle[1, 0] - w_left[0])
    ) / 4
    f_w_middle[-1, 0] -= Re * (
        psi[-2, 0] * (w_middle[-1, 1] - w_bottom[-1]) +
        psi[-1, 1] * (w_right[0] - w_middle[-2, 0])
    ) / 4
    f_w_middle[0, -1] += Re * (
        psi[1, -1] * (w_top[0] - w_middle[0, -2]) +
        psi[0, -2] * (w_middle[1, -1] - w_left[-1])
    ) / 4
    f_w_middle[-1, -1] -= Re * (
        psi[-2, -1] * (w_top[-1] - w_middle[-1, -2]) -
        psi[-1, -2] * (w_right[-1] - w_middle[-2, -1])
    ) / 4

    return np.concatenate([
        f_poisson, f_w_left, f_w_right, f_w_bottom, f_w_top, f_w_middle.flatten()
    ], axis=0)


def make_f(U_wall_top):
    return partial(f, U_wall_top=U_wall_top)


In [ ]:
def nonlinear_root_solver(f, N, Re, algorithm, **kwargs):
   
    solution = scipy.optimize.root(
        fun=f,
        x0=np.zeros(((N - 1) ** 2 + (N + 1) ** 2 - 4, )),
        method=algorithm,
        args=(
            Re, poisson_solvers.construct_laplacian_kernel_matrix(N=N-1, h=1),
        ),
        **kwargs,
    )

    psi, w = solution.x[:(N - 1) ** 2], solution.x[(N - 1) ** 2:]
    
    # Get final psi
    psi = psi.reshape(N - 1, N - 1)
    psi = np.pad(psi, (1, 1), mode="constant", constant_values=0)
    
    # Get final w
    w = streamfunction_vorticity_newton.reconstruct_w(w_tmp=w[:, None], N=N)
    w = w.reshape(N + 1, N + 1)
    
    return w, psi, solution

In [ ]:
%%time

N = 20
Re = 0 # i.e. viscosity mu = inf
U_wall_top = np.sin(np.pi * np.arange(1, N) / N) ** 2

w, psi, n_iter = streamfunction_vorticity_newton.newton_solver(
    f=partial(streamfunction_vorticity_newton.f, U_wall_top=U_wall_top),
    get_jacobian=streamfunction_vorticity_newton.get_jacobian, N=N, Re=Re,
)

fun = make_f(U_wall_top=U_wall_top)
jac = make_get_jacobian(f=fun)
options = {
    "line_search": None,
    "jac_options": {
        "reduction_method": "restart",
    },
}
w2, psi2, solution = nonlinear_root_solver(
    f=fun, N=N, Re=Re, algorithm="broyden2", options=options, tol=1e-10,
)
print(solution)

test_eq(np.allclose(w, w2), True)
test_eq(np.allclose(psi, psi2), True)
test_close(w, w2)
test_close(psi, psi2)
test_eq(n_iter, 1)

 message: A solution was found at the specified tolerance.
 success: True
  status: 1
     fun: [ 3.817e-11 -4.980e-11 ...  6.405e-10  1.455e-09]
       x: [ 2.137e-06 -2.671e-05 ...  1.121e+00  5.920e-01]
     nit: 307
CPU times: user 379 ms, sys: 7.62 ms, total: 387 ms
Wall time: 719 ms


In [ ]:
%%time

N = 20
Re = 10 # i.e. viscosity mu = inf
U_wall_top = np.sin(np.pi * np.arange(1, N) / N) ** 2

w, psi, n_iter = streamfunction_vorticity_newton.newton_solver(
    f=partial(streamfunction_vorticity_newton.f, U_wall_top=U_wall_top),
    get_jacobian=streamfunction_vorticity_newton.get_jacobian, N=N, Re=Re,
)

fun = make_f(U_wall_top=U_wall_top)
jac = make_get_jacobian(f=fun)
options = {
    "line_search": None,
    "jac_options": {
        "reduction_method": "restart",
        # "reduction_method": "svd",
    },
}
w2, psi2, solution = nonlinear_root_solver(
    f=fun, N=N, Re=Re, algorithm="broyden2", options=options, tol=1e-11,
)
print(solution)

# test_eq(np.allclose(w, w2, atol=1e-7), True)
test_eq(np.allclose(w, w2), True)
test_eq(np.allclose(psi, psi2), True)
test_close(w, w2, eps=1e-7)
test_close(psi, psi2, eps=1e-8)

 message: A solution was found at the specified tolerance.
 success: True
  status: 1
     fun: [-4.566e-12  1.965e-12 ... -8.263e-12  1.034e-10]
       x: [ 1.869e-06 -2.748e-05 ...  1.171e+00  6.297e-01]
     nit: 464
CPU times: user 885 ms, sys: 16.7 ms, total: 902 ms
Wall time: 1.02 s


In [ ]:
%%time

N = 40
Re = 10 # i.e. viscosity mu = inf
U_wall_top = np.sin(np.pi * np.arange(1, N) / N) ** 2

w, psi, n_iter = streamfunction_vorticity_newton.newton_solver(
    f=partial(streamfunction_vorticity_newton.f, U_wall_top=U_wall_top),
    get_jacobian=streamfunction_vorticity_newton.get_jacobian, N=N, Re=Re,
)

fun = make_f(U_wall_top=U_wall_top)
jac = make_get_jacobian(f=fun)
options = {
    "jac_options": {
        # "reduction_method": "restart",
        "reduction_method": "svd",
    },
}
w2, psi2, solution = nonlinear_root_solver(
    f=fun, N=N, Re=Re, algorithm="broyden2",
    tol=1e-8, # increasing this is too slow
    options=options, # chancing options does not help either
)
print(solution)

test_eq(np.allclose(w, w2), False)
test_eq(np.allclose(psi, psi2), False)

 message: A solution was found at the specified tolerance.
 success: True
  status: 1
     fun: [ 5.903e-04 -4.308e-04 ... -1.294e-02 -6.772e-02]
       x: [-1.148e-04  5.622e-05 ... -8.723e-01 -6.472e-01]
     nit: 134
CPU times: user 2.92 s, sys: 116 ms, total: 3.04 s
Wall time: 3.52 s


In [ ]:
#|eval: false
%%timeit
streamfunction_vorticity_newton.newton_solver(
    f=partial(streamfunction_vorticity_newton.f, U_wall_top=U_wall_top),
    get_jacobian=streamfunction_vorticity_newton.get_jacobian, N=N, Re=Re,
)

3.8 s ± 568 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Least squares

In [ ]:
def nonlinear_lstsq_solver(f, N, Re, algorithm, **kwargs):
   
    solution = scipy.optimize.least_squares(
        fun=f,
        x0=np.zeros(((N - 1) ** 2 + (N + 1) ** 2 - 4, )),
        method=algorithm,
        args=(
            Re, poisson_solvers.construct_laplacian_kernel_matrix(N=N-1, h=1),
        ),
        **kwargs,
    )

    psi, w = solution.x[:(N - 1) ** 2], solution.x[(N - 1) ** 2:]
    
    # Get final psi
    psi = psi.reshape(N - 1, N - 1)
    psi = np.pad(psi, (1, 1), mode="constant", constant_values=0)
    
    # Get final w
    w = streamfunction_vorticity_newton.reconstruct_w(w_tmp=w[:, None], N=N)
    w = w.reshape(N + 1, N + 1)
    
    return w, psi, solution

# Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()